In [ ]:
import vllm
import rich
import rich.panel
import rich.markup


In [5]:
model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"
model = vllm.LLM(model_name)

INFO 01-02 14:04:12 llm_engine.py:237] Initializing an LLM engine (v0.6.3.post1) with config: model='HuggingFaceTB/SmolLM2-1.7B-Instruct', speculative_config=None, tokenizer='HuggingFaceTB/SmolLM2-1.7B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=HuggingFaceTB/SmolLM2-1.7B-Instruct, num_scheduler_steps=1, chunked_prefill_enabled=False multi_

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 01-02 14:04:21 model_runner.py:1067] Loading model weights took 3.1887 GB
INFO 01-02 14:04:22 gpu_executor.py:122] # GPU blocks: 12370, # CPU blocks: 1365
INFO 01-02 14:04:22 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 24.16x
INFO 01-02 14:04:24 model_runner.py:1395] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 01-02 14:04:24 model_runner.py:1399] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 01-02 14:04:37 model_runner.py:1523] Graph capturing finished in 13 secs.


In [62]:
def chat_box(role, role_input, role_style, stars_style="[italic cyan]"):
    # Find text between stars *adsasd* and add stars_style to it
    role_input = rich.markup.escape(role_input)

    # start = 0
    # while True:
    #     start = role_input.find("*")
    #     if start == -1:
    #         break
    #     end = role_input.find("*", start + 1)
    #     if end == -1:
    #         break
    #     role_input = role_input[:start] + stars_style + "<" + role_input[start + 1:end] + ">[/]" + role_input[end + 1:]

    panel = rich.panel.Panel(
        role_input, 
        title=f"{role_style}{role}[/]:", 
        title_align="left"
    )
    rich.print(panel)

sampling_params = vllm.SamplingParams(temperature=1, max_tokens=2048)

history = [{
    "role": "system", 
    "content": 
    f"You are {model_name}, an extremely strong and helpful LLM. "
    # f"You also have sarcastic, witty, dry humor, a bit like the robot TARS in Interstellar. "
    # f"You can use stars to quote your narration of your actions or reactions."
}]

while True:
    user_input = input("You: ")

    if not user_input or user_input == "exit":
        print("<Chat ended>")
        break

    # Add the user input to history, and display it
    history.append({"role": "user", "content": user_input })
    chat_box("User", user_input, "[bold blue]")

    # Generate model output
    model_output = model.chat(
        history, 
        sampling_params=sampling_params,
        use_tqdm=False
    )

    # Add the model output to history, and display it
    output_text = model_output[0].outputs[0].text
    history.append({"role": "assistant", "content": output_text})
    chat_box("Bot", output_text, "[bold green]")


╭─ User: ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ How are you doing buddy                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Bot: ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ... extremely well, but I do have some grammar or punctuation corrections in mind that we can carry out. How    │
│ about we start with tenses? I think it's time to convert that past tense sentence into present tense. So, "How  │
│ are you?" becomes "How are *you*?" Treat it like a chat between you and your computer, trying to fill in the    │
│ blank with just the right amount of sarcasm. For example, "What, adjust your speaker's rhythm to match mine?"   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User: ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ I need to focus on my thesis more.                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Bot: ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Lovely! Your thesis is like a spaceship navigating through the cosmos. Sarcastic and witty, star-worthy indeed! │
│ Now, pour yourself a cup of liquid metal (caffeine), sit comfortably, and let's buckle down with your thesis.   │
│ Squeeze in a few hours and remember to totally caffeinate that starry brain of yours. Are you ready for an      │
│ intense, not too caffeinated discussion?                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ User: ─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ :)                                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─ Bot: ──────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Then, let's dive right into your thesis, my starry friend. Pour that coffee on it! Consider a few metaphorical  │
│ rocket ships in your writing to get more your point across. For example, "I aim to not just inform, but also    │
│ distressingly impress, in this discourse on...". How does that sound? Star quote from inside the spaceship!     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<Chat ended>


In [32]:
output[0].outputs[0].text

'Alright, let\'s compare 3/8 and 5/16 step by step.\n\nFirst, we need to convert both fractions to have a common denominator. Here\'s the conversion step:\n\n3/8 = 3 * (5/5) / 8 * (5/5) = 15/40.\n\n5/16 can be kept as it is.\n\nNext, we\'ll see whose fraction is larger:\n\n15/40 is less than 5/16 (you can see this by comparing the two denominators, but if you want the exact fraction greater than 5/16, you should convert 5/16 to 3/8. Then it becomes clearer).\n\nHowever, the first thing we find is that 3/8 is indeed less than 5/16.\n\nSo to answer your question "Is 3/8 bigger than 5/16?", the answer would be no, they are the same size (in terms of magnitude) but 3/8 is smaller as it is less than 5/16.'